In [1]:
import sys, os
sys.path.append('../MultiFishLSS/')
from headers import *
from twoPoint import *
from twoPointNoise import *
from classy import Class


## Generate $\sigma_8$ forecasts

#### Fiducial cosmology

In [2]:
default_cosmo = {
        'output': 'tCl lCl mPk',
        'non linear':'halofit',
        'l_max_scalars': 4000,
        'lensing': 'yes',
        'P_k_max_h/Mpc': 2.,
        'z_pk': '0.0,1087',
        'A_s': 2.10732e-9,
        'n_s': 0.96824,
        'alpha_s': 0.,
        'h': 0.6736,
        'N_ur': 2.0328,
        'N_ncdm': 1,
        'm_ncdm': 0.06,
        'tau_reio': 0.0544,
        'omega_b': 0.02237,
        'omega_cdm': 0.1200,
        'Omega_k': 0.,
        'Omega_Lambda': 0.,
        'w0_fld':-1,
        'wa_fld':0}
cosmo = Class()
cosmo.set(default_cosmo)
cosmo.compute()
h = cosmo.h()

chi = lambda zz: cosmo.comoving_distance(zz)*h

#### Experiment configuration

In [3]:
fsky = 5000/41253 # 5000 deg2

# two redshift bins from z = 2.7 to 3.3
zmin, zmax = 2.7, 3.3
nbins = 2

# 2 galaxy samples 
# name, bias, number density as lists
samples=['ga','gb']
b = [lambda z: 2.5, lambda z: 3.5] 
n = [lambda z: 2e-4, lambda z: 2e-4]

# overlap between stochastic terms
# index 0 and 2 are auto-correlations, so they must be 1
# index 1 is the cross-correlation between the two samples, set to 0 by default for no overlap
fover=[1,0,1]

# experiment name and basedir
bd='./'
surveyname='example'

exp = experiment(zmin=zmin, zmax=zmax, nbins=nbins, fsky=fsky, b=b, n=n,samples=samples,fover=fover)
forecast = fisherForecast(experiment=exp,cosmo=cosmo,name=surveyname,basedir=bd,ell=np.arange(10,3000,1))    

#### Compute derivatives (if not done already)

Derivatives are computed faster with dedicated node, using `example_setup.sh`


In [4]:
basis = np.array(['log(A_s)','b','N',])

forecast.free_params = basis
forecast.compute_derivatives() # will not overwrite unless specified

#### Use derivatives to construct Fisher matrix and get forecasts

$P_{gg}$

In [ ]:
tmp = []
for i in range(nbins):
    globe = 1
    z_i = forecast.experiment.zcenters[i]
    knl = 1/np.sqrt(forecast.Sigma2(z_i))
    # limit kmax to 0.1 h^-1 Mpc for linear regime
    F = forecast.gen_fisher(basis,globe,kmax_knl=.1/knl,zbins=np.array([i])) 
    Finv = np.linalg.inv(F)
    tmp.append(F)
    print(f'bin {i} at z={z_i:.2f}: sigma(s8)/s8 = {np.sqrt(np.diag(Finv))[0]/2:.4f}')
F = forecast.combine_fishers(tmp,1)
Finv = np.linalg.inv(F)
print(f'combined: sigma(s8)/s8 = {np.sqrt(np.diag(Finv))[0]/2:.4f}')


bin 0 at z=2.85: sigma(s8)/s8 = 0.0339
bin 1 at z=3.15: sigma(s8)/s8 = 0.0350
combined: sigma(s8)/s8 = 0.0244


$C^{\kappa g} + C^{gg}$ (assumes cross-correlation with SO by default)

In [16]:
tmp = []
for i in range(nbins):
    globe = 1
    z_i = forecast.experiment.zcenters[i]
    knl = 1/np.sqrt(forecast.Sigma2(z_i))
    F = forecast.gen_lensing_fisher(basis,globe,kmax_knl=.1/knl,bins=np.array([i]),kk=False,no_kg=False) 
    idxs = np.arange(globe).tolist()
    for j in range(F.shape[0]-globe): 
        if j%nbins == i: idxs += [globe + j] 
    tmp.append(F[idxs][:,idxs])
    Finv = np.linalg.inv(F[idxs][:,idxs]) # select only relevant parameters
    print('sigma (s8)/s8 at z=%.2f: %.4f'%(forecast.experiment.zcenters[i],np.sqrt(np.diag(Finv))[0]/2) )# ds8/s8 
F = forecast.combine_fishers(tmp,1)
Finv = np.linalg.inv(F)
print('combined sigma (s8)/s8: %.4f'%(np.sqrt(np.diag(Finv))[0]/2) )# ds8/s8

sigma (s8)/s8 at z=2.85: 0.0486
sigma (s8)/s8 at z=3.15: 0.0513
combined sigma (s8)/s8: 0.0353


$P_{gg} + C^{kg}$

In [ ]:
tmp = []
for i in range(nbins):
    globe = 1
    z_i = forecast.experiment.zcenters[i]
    knl = 1/np.sqrt(forecast.Sigma2(z_i))
    F = forecast.gen_fisher(basis,globe,kmax_knl=.1/knl,zbins=np.array([i]),kpar_min=1e-3) 
    F_Cl = forecast.gen_lensing_fisher(basis,globe,kmax_knl=.1/knl,bins=np.array([i]),kk=False,no_kg=False) 
    idxs = np.arange(globe).tolist()
    for j in range(F_Cl.shape[0]-globe): 
        if j%nbins == i: idxs += [globe + j] 
    F_Cl = F_Cl[idxs][:,idxs]

    # shared variables are logA + biases
    F = forecast.combine_2fishers(F,F_Cl,globe + len(forecast.experiment.samples) )
    tmp.append(F)
    Finv = np.linalg.inv(F)
    print('sigma (s8)/s8 at z=%.2f: %.4f'%(forecast.experiment.zcenters[i],np.sqrt(np.diag(Finv))[0]/2) )# ds8/s8 
F = forecast.combine_fishers(tmp,1)
Finv = np.linalg.inv(F)
print('combined sigma (s8)/s8: %.4f'%(np.sqrt(np.diag(Finv))[0]/2) )# ds8/s8